In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [2]:
######################
# Initiate Backtest  #
######################
from lib.multitasking_lib import *
cache_type ='backtest_uni'

my_cache = cache_state('backtest_uni')

def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("log/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9', mode='full'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler"], [kite_simulator, ohlc_tick_handler])
    
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


INFO:	Cache Pointing to: backtest_uni
INFO:	Cache Pointing to: backtest_uni
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: ohlc_tick_handler


True
['ohlc_tick_handlerlive', 'ohlc_tick_handlerbacktest_web', 'live_trade_handlerlive', 'kite_simulatorbacktest_web', 'order_handlerlive', 'freedom_initfreedom']
[<Process(Process-1, started)>]


DEBUG:	kite_simulator: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	ohlc_tick_handler: 1
DEBUG:	kite_simulator: {"stock": ["TCS"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1_new", "fromDate": "2019-4-8", "toDate": "2019-4-9", "freq": "1T", "mode": "full"}
DEBUG:	Trade_init: TCS
INFO:	Reset Cache for: TCSbacktest_uni
INFO:	TCSbacktest_uni=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day'}
INFO:	Running Full Backtest
DEBUG:	ohlc_tick_handler: start
TRADE: S: EN:  : 2019-04-08 09:18 : TCS -> 2053.0
TRADE: B: EX:  : 2019-04-08 09:20 : TCS -> 2057.55
TRADE: S: EN:  : 2019-04-08 09:24 : TCS -> 2059.75
TRADE: B: EX:  : 2019-04-08 09:25 : TCS -> 2061.0
TRADE: B: EN:  : 2019-04-08 0

In [3]:
backtest_unitest(['TCS'])

In [28]:
my_cache.get('TCSbacktest_uniTICK')

'{}'

In [ ]:
################################
###         Stop Trade       ###
################################

my_cache.delete('msgBufferQueue'+cache_postfix)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())

jobs
jobs = []

In [ ]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [ ]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

In [ ]:
cache_state('backtest_uni').getValue()

In [ ]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index[-1].minute+ohlc_data_temp.index[-1].hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

#sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
#print(trade_df1)
sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
buy = (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

In [ ]:
buy = pd.DataFrame()

if buy.shape[0] > 0 and buy[-1]==True:
    print('Yeah')

## Scratchpad

In [ ]:
redis_conn.pubsub_channels()

In [ ]:
my_cache = cache_state('backtest_uni')

In [ ]:
df = my_cache.getValue()

In [ ]:
pd.set_option('display.max_rows', 1000)
my_cache.getOHLC('TCS')

In [ ]:
trades_df = my_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

In [ ]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')
live_cache.getValue()
#live_cache.getOHLC('TCS', '1D').head()

In [1]:
from lib.data_model_lib import *
import json
live_cache = cache_state('live')

In [2]:
from kiteconnect import KiteConnect

In [4]:
kite_api_key = 'b2w0sfnr1zr92nxm'

live_cache.set('KiteAPIKey',kite_api_key)

True

In [5]:
live_cache.get('access_token')

'o2MI2Q7xSNPVSZOy24mQx5h28BL765Mv'

In [6]:
live_cache.get('KiteAPIKey')

'b2w0sfnr1zr92nxm'

In [99]:
KiteAPIKey = live_cache.get('KiteAPIKey')
kite = KiteConnect(api_key=KiteAPIKey)
access_token = live_cache.get('access_token')
kite.set_access_token(access_token)

In [100]:
kite.profile()

{'user_id': 'YM1755',
 'user_type': 'individual',
 'email': 'saswati.suhan@gmail.com',
 'user_name': 'Saswati Saha',
 'user_shortname': 'Saswati',
 'broker': 'ZERODHA',
 'exchanges': ['BSE', 'MCX', 'NSE', 'NFO', 'MF', 'CDS', 'BFO'],
 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'],
 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'],
 'avatar_url': '',
 'meta': {'demat_consent': 'physical'}}

In [ ]:
instruments_df = pd.DataFrame(kite.instruments())

In [157]:
cancel_order('CIPLA')

In [158]:
orders_df = pd.DataFrame(kite.orders())
orders_df[['order_id','status','tradingsymbol','transaction_type','quantity','tag','price']]

,order_id,status,tradingsymbol,transaction_type,quantity,tag,price
0,200512000670055,COMPLETE,WIPRO,SELL,1,freedom_v2,184.40
1,200512000683603,COMPLETE,WIPRO,BUY,1,freedom_v2,184.20
2,200512000803760,COMPLETE,WIPRO,BUY,1,freedom_v2,184.45
3,200512000782743,COMPLETE,WIPRO,BUY,1,freedom_v2,184.30
4,200512000988113,COMPLETE,WIPRO,SELL,2,None,0.00
5,200512000820643,COMPLETE,WIPRO,SELL,1,freedom_v2,185.70
6,200512001243114,COMPLETE,WIPRO,SELL,1,freedom_v2,186.55
7,200512001260328,COMPLETE,WIPRO,BUY,2,None,0.00
8,200512001267483,COMPLETE,CIPLA,SELL,1,freedom_v2,582.20
9,200512001288599,COMPLETE,WIPRO,BUY,1,freedom_v2,186.30


In [204]:
symbol = 'INFY'
live_cache.add(symbol)
token = int(live_cache.hmget('eq_token',symbol)[0])
live_cache.publish('live_trade_handlerlive', json.dumps({'cmd':'add','value':[token],'mode':'ltp'}))

live_cache.setValue('INFY','qty','1')
live_cache.setValue('INFY','SL %','0.4')
live_cache.setValue('INFY','TP %','1')
live_cache.setValue('INFY','algo','haikin_1_new')
live_cache.setValue('INFY','freq','1T')

live_cache.setValue('INFY','mode','live')

INFO:	INFYlive=>{'stock': 'INFY', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day', 'order_id': '0'}


1

1

In [203]:
live_cache.pubsub_channels()

['order_handlerlive',
 'kite_simulatorbacktest_web',
 'ohlc_tick_handlerlive',
 'freedom_initfreedom',
 'ohlc_tick_handlerbacktest_web',
 'live_trade_handlerlive']

In [217]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,CIPLA,1,0.4,1,574.65,0,-0.9500000000000455,-0.1650451702571309,-1.599999999999909,-0.27843034890801516,...,0.0,0,583.5,2020-05-12 05:48,SCANNING,PAUSE,haikin_1_new,1T,day,0
1,WIPRO,1,0.3,1,185.55,185.55,0,0,-1.849999999999909,-0.99703583939634,...,0.0,187.40550000000002,186.15,2020-05-12 07:03,SCANNING,live,haikin_1_new,1T,day,0
2,INFY,1,0.4,1,688.85,688.85,0,0,-2.4500000000000455,-0.35566523916673376,...,0.0,681.9615,690.0,2020-05-12 07:03,SHORT,paper,haikin_1_new,1T,day,0


In [215]:
live_cache.setValue('INFY','mode','paper')

0

In [209]:
live_cache.setValue('INFY','qty','1')
live_cache.setValue('INFY','SL %','0.4')
live_cache.setValue('INFY','TP %','1')
live_cache.setValue('INFY','algo','haikin_1_new')
live_cache.setValue('INFY','freq','1T')

0

In [198]:
live_cache.setValue('CIPLA','mode','PAUSE')

0

In [153]:
position_df = pd.DataFrame(kite.positions()['net'])
position_df

,tradingsymbol,exchange,instrument_token,product,quantity,overnight_quantity,multiplier,average_price,close_price,last_price,...,sell_quantity,sell_price,sell_value,sell_m2m,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value
0,WIPRO,NSE,969473,MIS,-1,0,1,185.75,0,186.55,...,8,185.750000,1486.0,1486.0,7,185.457143,1298.20,8,185.750000,1486.0
1,CIPLA,NSE,177665,MIS,0,0,1,0.00,0,577.75,...,3,580.933333,1742.8,1742.8,3,581.183333,1743.55,3,580.933333,1742.8


In [212]:
pd.DataFrame(kite.positions()['day'])

,tradingsymbol,exchange,instrument_token,product,quantity,overnight_quantity,multiplier,average_price,close_price,last_price,...,sell_quantity,sell_price,sell_value,sell_m2m,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value
0,WIPRO,NSE,969473,MIS,1,0,1,185.965385,0,186.5,...,12,185.9875,2231.85,2231.85,13,185.965385,2417.55,12,185.9875,2231.85
1,CIPLA,NSE,177665,MIS,0,0,1,0.000000,0,572.1,...,5,578.8300,2894.15,2894.15,5,579.110000,2895.55,5,578.8300,2894.15


In [196]:
orders_df = pd.DataFrame()
for msg in live_cache.xrange('notificationQueuelivenew'):
    tmp_df = pd.DataFrame.from_dict(json.loads(msg[1]['data']), orient='index').transpose()
    orders_df = orders_df.append(tmp_df)
    #print(msg)
orders_df

,placed_by,order_id,exchange_order_id,parent_order_id,status,status_message,status_message_raw,order_timestamp,exchange_update_timestamp,exchange_timestamp,...,market_protection,meta_raw,meta,tag,guid,account_id,unfilled_quantity,app_id,checksum,tags
0,YM1755,200512000988113,1300000003104710,None,OPEN,None,None,2020-05-12 10:02:20,2020-05-12 10:02:20,2020-05-12 10:02:20,...,0,None,{},None,01XY8cqQKrSEveK,YM1755,0,1,,NaN
0,YM1755,200512000988113,1300000003104710,None,COMPLETE,None,None,2020-05-12 10:02:20,2020-05-12 10:02:20,2020-05-12 10:02:20,...,0,None,{},None,01XY8cqQKrSEveK,YM1755,0,1,,NaN
0,YM1755,200512000820643,1300000002570977,None,COMPLETE,None,None,2020-05-12 10:07:41,2020-05-12 10:07:41,2020-05-12 09:51:06,...,0,None,{},freedom_v2,11327XrgOLOHNX7fOV,YM1755,0,11327,,[freedom_v2]
0,YM1755,200512001243114,1300000003903946,None,OPEN,None,None,2020-05-12 10:21:01,2020-05-12 10:21:01,2020-05-12 10:21:01,...,0,None,{},freedom_v2,11327X1Z4ldbX6OQO1,YM1755,0,11327,,[freedom_v2]
0,YM1755,200512001243114,1300000003903946,None,COMPLETE,None,None,2020-05-12 10:21:12,2020-05-12 10:21:12,2020-05-12 10:21:01,...,0,None,{},freedom_v2,11327X1Z4ldbX6OQO1,YM1755,0,11327,,[freedom_v2]
0,YM1755,200512001260328,1300000003948729,None,COMPLETE,None,None,2020-05-12 10:22:24,2020-05-12 10:22:24,2020-05-12 10:22:24,...,0,None,{},None,01Xh0OugN8LO12P,YM1755,0,1,,NaN
0,YM1755,200512001260328,1300000003948729,None,OPEN,None,None,2020-05-12 10:22:24,2020-05-12 10:22:24,2020-05-12 10:22:24,...,0,None,{},None,01Xh0OugN8LO12P,YM1755,0,1,,NaN
0,YM1755,200512001267483,1000000004408008,None,OPEN,None,None,2020-05-12 10:23:00,2020-05-12 10:23:00,2020-05-12 10:23:00,...,0,None,{},freedom_v2,11327XQ9t33v55698q,YM1755,0,11327,,[freedom_v2]
0,YM1755,200512001267483,1000000004408008,None,COMPLETE,None,None,2020-05-12 10:23:00,2020-05-12 10:23:00,2020-05-12 10:23:00,...,0,None,{},freedom_v2,11327XQ9t33v55698q,YM1755,0,11327,,[freedom_v2]
0,YM1755,200512001288599,1300000004034075,None,COMPLETE,None,None,2020-05-12 10:25:00,2020-05-12 10:25:00,2020-05-12 10:25:00,...,0,None,{},freedom_v2,11327XJl5l1tpNXt8X,YM1755,0,11327,,[freedom_v2]


In [ ]:
instruments_df.to_hdf('data/instruments.hdf',key='instruments')

In [ ]:
instruments_df.columns

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

In [ ]:
kite.margins()

In [21]:
#live_cache.publish('live_trade_handlerlive','profile')
live_cache.publish('live_trade_handlerlive','status')

1

In [140]:
live_cache.publish('live_trade_handlerlive','INIT')

1

In [25]:
live_cache.publish('live_trade_handlerlive','CLOSE')

1

In [50]:
live_cache.publish('live_trade_handlerlive','START')

1

In [141]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'add','value':[177665, 969473],'mode':'ltp'}))

1

In [ ]:
import json

In [28]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'remove','value':[969473, 2953217],'mode':'quote'}))

1

In [ ]:
live_cache.publish('live_trade_handlerlive','add')

In [84]:
live_cache.publish('order_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':184.45,'qty':1}))

1

In [85]:
live_cache.publish('order_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':185.7,'qty':1}))

1

In [ ]:
buy_limit('WIPRO',186.5,1)

In [ ]:
sell_limit('WIPRO',187.2,1)

In [151]:
live_cache.getOHLC('CIPLA')

,close,high,low,open
2020-05-12 03:36:00,586.05,586.05,586.05,586.05
2020-05-12 03:37:00,587.00,587.00,586.05,586.05
2020-05-12 03:53:00,593.25,593.25,593.25,593.25
2020-05-12 04:04:00,587.85,587.85,587.55,587.60
2020-05-12 04:05:00,587.45,588.20,587.20,588.00
...,...,...,...,...
2020-05-12 05:27:00,579.00,579.00,578.40,578.45
2020-05-12 05:28:00,578.35,579.00,578.10,579.00
2020-05-12 05:29:00,577.95,578.35,577.50,578.30
2020-05-12 05:30:00,577.80,578.45,577.70,577.95


In [115]:
live_cache.setValue('WIPRO','state','SCANNING')

0

In [150]:
live_cache.get('last_id_msg')

'1589261497811-0'

In [ ]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [ ]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


In [ ]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

In [ ]:
live_cache.delete('eq_token')

In [ ]:
eq_nse.head().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

In [ ]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

In [ ]:
live_cache.hgetall('eq_token')

In [ ]:
live_cache.hget('eq_token',[2953217, 969473])

In [ ]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [ ]:
msg['data']

In [ ]:
val = json.loads(msg['data'])

In [ ]:
for data in val:
    instrument_token = data['instrument_token']
    stock_id = live_cache.hmget('eq_token',instrument_token)[0]


    ltp = data['last_price']
    #temp_df = pd.DataFrame(data={'date':[date_val],'ltp':[ltp]})
    #temp_df = temp_df.set_index('date')
    #temp_df.index = pd.to_datetime(temp_df.index)
    print(stock_id)
    print(ltp)
    hash_key = stock_id

In [ ]:
stock_id

In [ ]:
live_cache.getValue()

In [ ]:
live_cache.add('WIPRO')

In [ ]:
stock_key = 'TCS'
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')

In [ ]:
live_cache.add('TCS')

In [ ]:
live_cache.remove()

In [ ]:
live_cache.getValue()

In [52]:
live_cache.hmget('eq_token',['CIPLA','WIPRO'])

['177665', '969473']

In [39]:
from lib.data_model_lib import *

In [14]:
live_cache = cache_state('backtest_web')

In [30]:
live_cache.hmget('eq_token',['TCS','WIPRO','AXISBANK'])

['2953217', '969473', '1510401']

In [64]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,CIPLA,1,0.4,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,593.25,2020-05-12 04:10,SCANNING,paper,haikin_1_new,1T,day,0
1,WIPRO,1,0.3,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,185.6,2020-05-12 04:10,SCANNING,live,haikin_1_new,1T,day,0


In [89]:
live_cache.getOHLC('WIPRO')

,close,high,low,open
2020-05-12 04:04:00,185.40,185.40,185.40,185.40
2020-05-12 04:05:00,185.40,185.45,185.35,185.40
2020-05-12 04:06:00,185.40,185.60,185.20,185.45
2020-05-12 04:07:00,184.85,185.45,184.85,185.40
2020-05-12 04:08:00,184.90,185.00,184.70,184.85
2020-05-12 04:09:00,184.90,185.00,184.75,184.90
2020-05-12 04:10:00,184.85,184.95,184.70,184.90
2020-05-12 04:11:00,184.90,184.90,184.75,184.85


In [56]:
live_cache.xrange('msgBufferQueuelive')

[('1589255605992-0',
  {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 177665, "last_price": 593.25}, {"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 184.7}]'}),
 ('1589255607218-0',
  {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 184.7}, {"tradable": true, "mode": "ltp", "instrument_token": 177665, "last_price": 593.2}]'}),
 ('1589255608450-0',
  {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 177665, "last_price": 593.0}]'}),
 ('1589255609020-0',
  {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 177665, "last_price": 593.1}]'}),
 ('1589255609375-0',
  {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 177665, "last_price": 592.85}, {"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 184.7}]'}),
 ('1589255609986-0',
  {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 177665, "last_price": 592.5}]'}),
 ('15892556108

In [8]:
live_cache.sadd('ticker_list','2953217')
live_cache.sadd('ticker_list','969473')

live_cache.sadd('ticker_list','1510401')

1

In [9]:
list(live_cache.smembers('ticker_list'))

['969473', '2953217', '1510401']

In [51]:
live_cache.srem('ticker_list','WIPRO')

0

In [54]:
datetime.fromtimestamp(1589255638249/1000)

datetime.datetime(2020, 5, 12, 3, 53, 58, 249000)